<a href="https://colab.research.google.com/github/Anjasfedo/Learning-TensorFlow/blob/main/learn_by_doing/Police_Arrest_from_2021_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download datasets

In [2]:
!pip install kaggle

from google.colab import files
files.upload

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d shayanshahid997/police-arrest-from-2021-2023

!unzip police-arrest-from-2021-2023.zip

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/shayanshahid997/police-arrest-from-2021-2023
License(s): CC0-1.0
police-arrest-from-2021-2023.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  police-arrest-from-2021-2023.zip
  inflating: Police_Arrests_20240702.csv  


# Initialize Packages

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models

# Preprocessing Dataset

In [8]:
df = pd.read_csv('Police_Arrests_20240702.csv')
df

,ID Reference Number,Subject's race,Subject's gender,Subject's age,Ethnicity,District of occurrence,Adjacent to School,Assigned Division,Assigned Bureau,Event Date/Time
0,1,Asian,Male,43,NON-HISPANIC,6.0,0,TOD,FSB,1/1/2021 1:07
1,2,Black/African American,Male,23,NON-HISPANIC,1.0,0,RCPD,NaN,1/1/2021 0:52
2,3,Black/African American,Male,18,NON-HISPANIC,1.0,0,RCPD,NaN,1/1/2021 0:52
3,4,Black/African American,Male,21,NON-HISPANIC,1.0,0,RCPD,NaN,1/1/2021 0:52
4,5,White,Female,38,HISPANIC,4.0,0,TOD,FSB,1/1/2021 3:00
...,...,...,...,...,...,...,...,...,...,...
23505,23506,Black/African American,Male,30,NON-HISPANIC,3.0,0,3D,PSB,6/30/2024 11:00
23506,23507,Asian,Male,60,NON-HISPANIC,3.0,0,3D,PSB,7/1/2024 7:27
23507,23508,White,Male,22,HISPANIC,4.0,0,2D,PSB,6/29/2024 2:42
23508,23509,White,Male,25,HISPANIC,4.0,0,2D,PSB,6/29/2024 2:42


In [9]:
df.isnull().sum()

ID Reference Number          0
Subject's race               0
Subject's gender             0
Subject's age                0
Ethnicity                    0
District of occurrence     102
Adjacent to School           0
Assigned Division          129
Assigned Bureau           4659
Event Date/Time              0
dtype: int64

Column information:
- ID Reference Number          : unique identifier for each arrest record
- Subject's race               : demographic information
- Subject's gender             : demographic information
- Subject's age                : demographic information
- Ethnicity                    : demographic information
- District of occurrence       : indicates the district in which the arrest took place (missing)
- Adjacent to School           : whether the arrest took place within 500 feet of a school, with a 1 indicating that it did and a 0 indicating that it did not
- Assigned Division            : division and bureau of the police department that were assigned to the arrest (missing)
- Assigned Bureau              : division and bureau of the police department that were assigned to the arrest (missing)
- Event Date/Time              : provides the date and time of the arrest